In [1]:
import time
import random
import operator
import numpy as np
import pandas as pd
# # from binarytree import Node,build
# import random
# from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
import operator

import deap
from deap import base
from deap import creator
from deap.gp import *
from deap.tools import *





TEMPERATURE = 1

Pc = .95
Pm = .05

NO_OF_GENERATIONS = 100

N_ROWS = 10
N_COLUMNS = 10


P_CORRELATION_BOUND = .7   #TODO: RELATE WITH TEMPERATURE


avg_pearson_check_time = time.time()
avg_fitness_calculation_time = time.time()


def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

    
# GP Expression Trees
pset = PrimitiveSet("main", 9)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
# pset.addEphemeralConstant("randconst", lambda: random.randint(-1,1))

pset.renameArguments(ARG0="thickness")
pset.renameArguments(ARG1="size")
pset.renameArguments(ARG2="shape")
pset.renameArguments(ARG3="ma")
pset.renameArguments(ARG4="se")
pset.renameArguments(ARG5="bn")
pset.renameArguments(ARG6="bc")
pset.renameArguments(ARG7="nn")
pset.renameArguments(ARG8="mitoses")


# expr = genFull(pset, min_=1, max_=5)
# tree = PrimitiveTree(expr)
# type(tree) == PrimitiveTree




creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", PrimitiveTree, fitness=creator.FitnessMin, pset=pset, batchfitness=[], temp=TEMPERATURE)

toolbox = base.Toolbox()
toolbox.register("expr", genFull, pset=pset, min_=3, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)

# toolbox.register("row", tools.initRepeat, list, toolbox.individual, N_ROWS)
# toolbox.register("population", tools.initRepeat, list, toolbox.row, N_COLUMNS)





# type(individual) = deap.creator.Individual
# inner_grid = np.array(population)
 
grid = np.ndarray(shape=(N_ROWS + 2, N_COLUMNS + 2), dtype=deap.creator.Individual)

def wrapGrid():
    inner_grid = grid[1:-1,1:-1]
    grid[1:-1, 1:-1] = inner_grid
    grid[:1,1:-1] = inner_grid[-1]
    grid[-1:,1:-1] = inner_grid[0]
    grid[1:-1,-1:] = inner_grid[:,:1]
    grid[1:-1,:1] = inner_grid[:,-1:]
    grid[0][0] = inner_grid[-1][-1]
    grid[-1][-1] = inner_grid[0][0]
    grid[0][-1] = inner_grid[-1][0]
    grid[-1][0] = inner_grid[0][-1]


# print([[type(grid[i][j]) for j in range(len(grid[0]))] for i in range(len(grid))])

toolbox.register("compile", compile, pset=pset)

C:\Users\lethalTranquilizer\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [2]:
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

def makeBatches():
    data = pd.read_csv('Dataset_MLAI.txt',sep = ',',header = None)
    #display(data)
 
    data[6] =  pd.to_numeric(data[6], errors='coerce') 
    # drop nas
    data = data.dropna()


    x_features = data.iloc[:,1:10]
    y_features = data.iloc[:,10]
    y_list = list(y_features.values.T.ravel())
    x_array = x_features.values

    scaler = preprocessing.StandardScaler()
    scaled_x_l = []
    y_train_l = []
    for i in range(12):
        x_train, x_test, y_train, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state= random.randint(1,100),shuffle = True)
        scaled_x_l.append(scaler.fit_transform(x_train))
        y_train_l.append(y_train)
    return(scaled_x_l,y_train_l)

scaled_x_lit,y_train_lit = makeBatches()
# print(scaled_x_lit, y_train_lit)


C:\Users\lethalTranquilizer\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\lethalTranquilizer\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\lethalTranquilizer\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [3]:
def treeBatchFitness(tree, x=scaled_x_lit, y=y_train_lit):
    function = compile(str(tree), pset)
    fitnessS=[]
    scaled_x_lit,y_train_lit = x,y
    for j in range(12):
        tn=0
        tp=0
        fn=0
        fp=0
        listRand=[]
        for i in range(30):
            if(function((scaled_x_lit[j])[i,0],(scaled_x_lit[j])[i,1],(scaled_x_lit[j])[i,2],(scaled_x_lit[j])[i,3],(scaled_x_lit[j])[i,4],(scaled_x_lit[j])[i,5],(scaled_x_lit[j])[i,6],(scaled_x_lit[j])[i,7],(scaled_x_lit[j])[i,8])<0):
                listRand.append(2)
            else:
                listRand.append(4)
#         print("Batch "+str(j+2)+" : ")

        for n in range(30):
            if(listRand[n]==(y_train_lit[j])[n]):
                if((y_train_lit[j])[n]==2):
                    tn=tn+1
                else:
                    tp=tp+1
            else:
                if(listRand[n]==2):
                    fn=fn+1
                else:
                    fp=fp+1
#         print("FP : "+str(fp)+" \n "+ "FN : "+str(fn)+" \n "+ "TP : "+str(tp)+" \n "+"TN : "+str(tn)+" \n ")
        if tp!= 0:
            prec=tp/(tp+fp) if fp!= 0 else 0
            recall=tp/(tp+fn) if fn!= 0 else 0
            fitness=(2*prec*recall)/(prec+recall) if prec != 0 and recall != 0 else 0
        else:
            fitness = 0
        fitnessS.append(fitness*100)
    return(fitnessS)

In [4]:
def pearsonEvaluation(tree1 ,tree2):
    lsum_product = []; lsumx_product = [];  lsumy_product = [];
#     f_tree1 = Tree_fitness(tree1,scaled_x_lit,y_train_lit)
#     f_tree2 = Tree_fitness(tree2,scaled_x_lit,y_train_lit)
    f_tree1 = tree1.batchfitness
    f_tree2 = tree2.batchfitness
    x_mean = np.mean(f_tree1)
    y_mean = np.mean(f_tree2)
    for j,k in zip(f_tree1,f_tree2):
        sub_x = j-x_mean; sub_y = k-y_mean
        product = (sub_x)*(sub_y)
        square_x = sub_x**2;square_y = sub_y**2

        lsum_product.append(product); lsumx_product.append(square_x); lsumy_product.append(square_y);
    x = np.sum(lsum_product)
    y = np.sqrt(np.sum(lsumx_product))
    z = np.sqrt(np.sum(lsumy_product))
#     print(str(tree1))
#     print(str(tree2))
    return( (x/(y*z))**2)
    
# Pearson_evaluation(x[2][2],x[2][2])


In [5]:
def pearsonCheck(tree1, tree2):
    return pearsonEvaluation(tree1, tree2)
# def main_fitness_update(indicies):




def generateTree():
    ind = toolbox.individual()
    ind.batchfitness = treeBatchFitness(ind)
    ind.fitness.values = (np.mean(ind.batchfitness),)
    return ind
     
def fitness_update_to_grid(indicies):
    batch_fitnesses = treeBatchFitness(grid[indicies[0]][indicies[1]])
    grid[indicies[0]][indicies[1]].batchfitness = batch_fitnesses
    grid[indicies[0]][indicies[1]].fitness.values= (np.mean(batch_fitnesses),)
    # should simultaneously update batch fitnesses
    


def checkHood(new_tree, location, temp=TEMPERATURE):
    for i in range(location[0]-1, location[0] + 2):
        for j in range(location[1] - 1, location[1] + 2):
            if (i,j) != tuple(location):
                if type(grid[i][j]) == deap.creator.Individual: # neighbour presence check
                    if pearsonCheck(grid[i][j], new_tree) > float(P_CORRELATION_BOUND * (1/temp)): # pearson check 
                        return 0
    return 1


def initializePOPtoGrid():
    fillq = [[i,j] for i in range(1, N_ROWS + 1) for j in range(1, N_COLUMNS + 1)]
    random.shuffle(fillq)
    
    for location in fillq:
        sapling = generateTree()
        while checkHood(sapling, location) != 1:
            sapling = generateTree()
#         print(location)
        grid[location[0]][location[1]] = sapling
        #wrapGrid everytime because new saplings may need these neighbours
        if location[0] in (1, N_ROWS) or location[1] in (1, N_COLUMNS):
            wrapGrid()
#         print([[type(grid[i][j]) for j in range(len(grid[0]))] for i in range(len(grid))])    

#     for i in range(len(grid)):
#         for j in range(len(grid[0])):
#             fitness_update_to_grid((ij))

initializePOPtoGrid()
# ind1 = generateTree()
# ind2 = generateTree()
# ind1.temp = 0.95
# ind2.temp = 0.90
# p1, p2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
# children = deap.gp.cxOnePoint(p1, p2)
# for child in children:
#     print(child.temp)
    
# print([[type(grid[i][j]) for j in range(len(grid[0]))] for i in range(len(grid))])    

C:\Users\lethalTranquilizer\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in double_scalars
<string>:1: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\lethalTranquilizer\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\lethalTranquilizer\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars


In [7]:
def pickFittestNeighbour(location, avoid):
    fittest = (location[0] - 1, location[1] - 1)
    for i in range(location[0]-1, location[0] + 2):
        for j in range(location[1] - 1, location[1] + 2):
#             print(i,j)
            if (i,j) not in avoid and grid[i][j].fitness.values[0] > grid[fittest[0]][fittest[1]].fitness.values[0]:
                fittest = (i,j) 
    return fittest




def performCross(location, neighbour):
    ind1 = grid[location[0]][location[1]]
    ind2 = grid[neighbour[0]][neighbour[1]]
    p1, p2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
    children = deap.gp.cxOnePoint(p1, p2)
    for child in children:
        child.batchfitness = treeBatchFitness(child)
        child.fitness.values = (np.mean(child.batchfitness),)
    return children[0] if children[0].fitness.values[0] > children[1].fitness.values[0] else children[1]

    

    

operator_matrix = [['c' for i in range(N_ROWS + 2)] for j in range(N_COLUMNS + 2)]
temperature_matrix = [[0.0 for i in range(N_ROWS + 2)] for j in range(N_COLUMNS + 2)]
fitness_matrix = [[0 for i in range(N_ROWS + 2)] for j in range(N_COLUMNS + 2)]



max_fitness = ((0,0), 0.0)

if __name__ == '__main__':
    new_gen_pop = np.ndarray(shape=(N_ROWS + 2, N_COLUMNS + 2), dtype=deap.creator.Individual)
    for gen in range(NO_OF_GENERATIONS):
        print('#GEN', gen)
        # GENERATION ROUTINE
        # TODO: EVERY CELL CAN BE THREADED
        for i in range(1, N_ROWS + 1):
            for j in range(1, N_COLUMNS + 1):
                dice = np.random.random()
                # crossover block
                if dice < Pc:
                    for trials in range(8):
                        avoid = []
    #                     print(avoid)
                        neighbour = pickFittestNeighbour((i,j), avoid)
                        child = performCross((i,j), neighbour)
                        # 
#                         if (child.fitness.values[0] > grid[i][j].fitness.values[0] and checkHood(child, (i,j))) or np.random.random() < grid[i][j].temp:
                        if child.fitness.values[0] > grid[i][j].fitness.values[0] or np.random.random() < grid[i][j].temp:
    #                   print(type(child))
                            new_gen_pop[i][j] = child
                            # TODO: similarity check for child passing 
                            operator_matrix[i][j] = 'c'
                            break
                        else:
                            avoid.append(neighbour)
                            continue
                    new_gen_pop[i][j] = grid[i][j] # reproduction if no fitter individual is found
                    operator_matrix[i][j] = 'cr'
                # mutation block 
                elif dice < Pc + Pm:
                    # mutate sub tree then update fitness
                    mutate = toolbox.clone(grid[i][j])
                    mutated = mutUniform(mutate, toolbox.expr, pset=pset)[0]
                    mutated.batchfitness = treeBatchFitness(mutate)
                    mutated.fitness.values = (np.mean(mutate.batchfitness),)
                    new_gen_pop[i][j] =  mutated
                    operator_matrix[i][j] = 'm'
                # reproduction block
                else: 
                    new_gen_pop[i][j] = grid[i][j]
                    operator_matrix[i][j] = 'r'
        print('operator matrix\n', operator_matrix)

        # temperature updates
        for i in range(1, N_ROWS + 1):
            for j in range(1, N_COLUMNS + 1):
                alpha = (9.9 + (np.random.random() / 10)) / 10
                grid[i][j].temp = grid[i][j].temp * alpha
                temperature_matrix[i][j] = round(grid[i][j].temp, 3)

        print('temperature matrix\n', temperature_matrix)
#         genr = [print('#GEN', gen) for gen in range(NO_OF_GENERATIONS)]
        # max fitness search
        # construct fitness matrix for verbosity
        for i in range(1, N_ROWS + 1):
            for j in range(1, N_COLUMNS + 1):
                cursor = new_gen_pop[i][j]
                max_fitness =  ((i,j), cursor.fitness.values[0]) if max_fitness[1] < cursor.fitness.values[0] else max_fitness
                fitness_matrix[i][j] = round(cursor.fitness.values[0], 3)
        print('fitness_matrix\n', fitness_matrix)
        print('Gen', gen, 'max fitness', max_fitness)
        if max_fitness[1] > 90:
            print('optimal fitness achieved, halting...')
            break

        # reinitialize grids
        grid = new_gen_pop
        wrapGrid()
        new_gen_pop = np.ndarray(shape=(N_ROWS + 2, N_COLUMNS + 2), dtype=deap.creator.Individual)




























#GEN 0


C:\Users\lethalTranquilizer\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in double_scalars
<string>:1: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\lethalTranquilizer\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars


operator matrix
 [['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'm', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'm', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'm', 'm', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'm', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'm', 'cr', 'm', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c']]
temperature matrix
 [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.994, 0.994, 0.999, 0.994, 0.991, 0.995, 0.995, 0.998, 

operator matrix
 [['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'm', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'm', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'm', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'm', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'm', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c']]
temperature matrix
 [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.978, 0.974, 0.982, 0.977, 0.965, 0.98, 0.977, 0.983,

operator matrix
 [['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'm', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'm', 'm', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c']]
temperature matrix
 [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.966, 0.958, 0.97, 0.962, 0.951, 0.974, 0.968, 0.96

operator matrix
 [['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'm', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'm', 'cr', 'cr', 'cr', 'm', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'm', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'm', 'c'], ['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c']]
temperature matrix
 [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.951, 0.938, 0.955, 0.944, 0.94, 0.969, 0.957, 0.963,

operator matrix
 [['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'm', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'm', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'm', 'cr', 'c'], ['c', 'cr', 'cr', 'm', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'm', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'm', 'm', 'cr', 'm', 'cr', 'cr', 'cr', 'cr', 'm', 'cr', 'c'], ['c', 'cr', 'm', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'cr', 'c'], ['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c']]
temperature matrix
 [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.937, 0.915, 0.945, 0.928, 0.933, 0.953, 0.948, 0.955, 0.9

KeyboardInterrupt: 

In [ ]:
# grid = np.array([[0 for i in range(10)] for j in range(10)])
# for i in range(1, 8 + 1):
#     for j in range(1, 8 + 1):
#         grid[i][j] = np.random.randint(1,10)
# print(grid)
# print()
# wrapGrid()
# print(grid)
print(round(0.9999999, 2))
print(9.9 + (np.random.random() / 10)) / 10
